In [ ]:
! pip install -U -q sodapy confluent-kafka pandas 

wget https://dlcdn.apache.org/kafka/3.7.0/kafka_2.13-3.7.0.tgz
tar -xzf kafka_2.13-3.7.0.tgz
mv kafka_2.13-3.7.0  kafka
cd kafka

bin/zookeeper-server-start.sh config/zookeeper.properties
bin/kafka-server-start.sh config/server.properties

bin/kafka-topics.sh --create --topic service-requests-events --bootstrap-server localhost:9092
bin/kafka-console-consumer.sh --topic service-requests-events --from-beginning --bootstrap-server localhost:9092

In [ ]:
from confluent_kafka import Producer
import pandas as pd
import json
from sodapy import Socrata
import time

In [ ]:
data_url = "data.cityofnewyork.us"
data_set = "erm2-nwe9"
app_token = "tEeu8oBfckAQMIatiKafE7oeO"
bootstrap_servers = "localhost:9092"
topic = "service-requests-events"

In [ ]:
client = Socrata(data_url, app_token)
producer = Producer({"bootstrap.servers": bootstrap_servers})
client.timeout = 60
offset = 0
api_limit = 100

In [ ]:

def send_data_to_kafka(api_offset):
    try:
        results = client.get(data_set, limit=api_limit, offset=api_offset)
        df = pd.DataFrame.from_records(results)

        for _, row in df.iterrows():
            producer.produce(topic, row.to_json())

        producer.flush()

        print(f"Data sent to Kafka topic successfully. {df.shape}")
    except Exception as ex:
        print(ex)

In [ ]:
while True:
    send_data_to_kafka(offset)
    time.sleep(5)
    offset = offset + api_limit